<a href="https://colab.research.google.com/github/ai-for-dld/ai_for_dld_udemy/blob/main/colab/ai_for_dld_0401_ai_seq_tb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Debugging HDL using AI explanations – Compile, Identify, and Fix Errors

# Setting Up Your Digital Logic Simulation Environment in Google Colab

This chapter provides a practical guide to configuring a Google Colab environment for simulating digital logic circuits using Icarus Verilog and GHDL. Google Colab, a cloud-based Jupyter notebook environment, offers a convenient way to run HDL simulations without local installations, making it ideal for learning, experimentation, and collaborative projects.

## Why Colab for HDL Simulation?

Google Colab provides a free, pre-configured environment with access to GPUs (though not necessary for basic HDL simulation) and a robust Linux backend. This eliminates the complexities of setting up toolchains locally, dealing with operating system compatibility issues, or managing dependencies. It's particularly beneficial for:

* **Learning:** Quickly get started with HDL coding and simulation without extensive setup.
* **Collaboration:** Share Colab notebooks with code and simulation results, fostering easy collaboration.
* **Accessibility:** Work on your designs from any device with an internet connection.
* **Reproducibility:** Ensure your simulation environment is consistent across different sessions and users.

## Colab-Compatible `%%bash` Script for Tool Installation

The following `%%bash` script is designed to be run directly within a Google Colab cell. It handles the installation of Icarus Verilog (for Verilog simulation) and GHDL (for VHDL simulation), along with necessary system updates.


In [ ]:
%%bash
# Update the package list to ensure we get the latest versions of packages.
# The -qq flag makes apt update run silently.
echo "Updating apt package list..."
sudo apt update -qq

# Install Icarus Verilog, a popular open-source Verilog compiler and simulator.
# The -y flag automatically confirms installation prompts.
# The -qq flag makes apt install run silently.
echo "Installing Icarus Verilog..."
sudo apt install -y -qq iverilog

# Install GHDL, an open-source VHDL simulator that supports VHDL-87, VHDL-93, and VHDL-2002.
# The -y flag automatically confirms installation prompts.
# The -qq flag makes apt install run silently.
echo "Installing GHDL..."
sudo apt install -y -qq ghdl

# Verify the installation of Icarus Verilog by checking its version.
echo "Verifying Icarus Verilog installation..."
iverilog -V

# Verify the installation of GHDL by checking its version.
echo "Verifying GHDL installation..."
ghdl --version

# Print a success message if all installations and verifications complete without error.
echo "✅ All tools installed successfully!"


Updating apt package list...
36 packages can be upgraded. Run 'apt list --upgradable' to see them.
Installing Icarus Verilog...
Suggested packages:
  gtkwave
The following NEW packages will be installed:
  iverilog
0 upgraded, 1 newly installed, 0 to remove and 36 not upgraded.
Need to get 2,130 kB of archives.
After this operation, 6,749 kB of additional disk space will be used.
Selecting previously unselected package iverilog.
(Reading database ... 126284 files and directories currently installed.)
Preparing to unpack .../iverilog_11.0-1.1_amd64.deb ...
Unpacking iverilog (11.0-1.1) ...
Setting up iverilog (11.0-1.1) ...
Processing triggers for man-db (2.10.2-1) ...
Installing GHDL...
The following additional packages will be installed:
  gcc-10-base ghdl-common ghdl-mcode libgnat-10
Suggested packages:
  gtkwave
The following NEW packages will be installed:
  gcc-10-base ghdl ghdl-common ghdl-mcode libgnat-10
0 upgraded, 5 newly installed, 0 to remove and 36 not upgraded.
Need to ge



W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 1.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tty.)
debconf: falling back to frontend: Teletype
dpkg-preconfigure: unable to re-open stdin: 


debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 5.)
debconf: falling back to frontend: Readline
debconf: unable to initialize frontend: Readline
debconf: (This frontend requires a controlling tt

### Verifying the Installation

After running the script, the `iverilog -V` and `ghdl --version` commands will display the installed versions of the respective tools. This confirms that the executables are available in your Colab environment's PATH and are ready for use.

### Example Output Snippet (versions may vary):

```
Icarus Verilog version 11.0 (stable)
(c) 2002-2022 Stephen Williams <steve@icarus.com>
...

GHDL 2.0.0 (Ubuntu 22.04.4 LTS)
Compiled with GNAT 11.4.0
License: GNU GPL. See the COPYING file for more details.

✅ All tools installed successfully!
```

With these tools successfully installed, your Google Colab environment is now ready for compiling and simulating your Verilog and VHDL digital logic designs. You can proceed to write your HDL code in subsequent cells, compile it using `iverilog` or `ghdl`, and then run the generated simulation executables.

# Essential Sequential Logic Circuits: RTL Implementation in Verilog and VHDL

This chapter delves into the Register Transfer Level (RTL) implementation of fundamental sequential logic circuits using both Verilog and VHDL. Sequential circuits, unlike combinational circuits, have memory and their outputs depend not only on current inputs but also on past inputs (i.e., their current state). This memory element is typically realized using flip-flops.

For each circuit, we will provide RTL code that incorporates standard clock and reset inputs, crucial for synchronous digital system design. The `%%writefile` magic command is used to directly save the code into files within a Jupyter (or Google Colab) environment, facilitating easy simulation and synthesis later.

## Common Features: Clock and Reset

Throughout these examples, `clk` (clock) and `rst` (reset) are standard 1-bit inputs.

* **`clk`**: The clock signal dictates the timing of state changes in synchronous circuits.
    Most circuits will be positive-edge triggered, meaning state updates occur on the rising edge of `clk`.
* **`rst`**: The reset signal is used to bring the circuit to a known initial state.
    For simplicity, we will implement an **asynchronous active-high reset** in these examples.
    This means when `rst` is high, the circuit immediately goes to its reset state, regardless of the clock.

## 1. T Flip-Flop

The T (Toggle) flip-flop is a single-input version of the JK flip-flop. When the T input is high, the flip-flop toggles its output on the active clock edge. When T is low, the output remains unchanged.

### Verilog RTL for T Flip-Flop


In [ ]:
%%writefile t_ff.v
// Module: t_ff
// Description: T (Toggle) Flip-Flop with asynchronous active-high reset.

module t_ff (
    input clk,  // Clock input (positive-edge triggered)
    input rst,  // Asynchronous active-high reset
    input T,    // Toggle input
    output reg Q // Output of the T Flip-Flop
);

    // Always block sensitive to positive clock edge or positive reset edge
    always @(posedge clk or posedge rst) begin
        if (rst) begin
            // Asynchronous reset: Q is cleared to 0
            Q <= 1'b0;
        end else begin
            // Synchronous operation:
            if (T) begin
                // If T is high, toggle Q
                Q <= ~Q;
            end
            // If T is low, Q retains its current value (Q <= Q is implicit)
        end
    end

endmodule

Writing t_ff.v


### VHDL RTL for T Flip-Flop


In [ ]:
%%writefile t_ff.vhdl
-- Entity: t_ff
-- Description: T (Toggle) Flip-Flop with asynchronous active-high reset.

library ieee;
use ieee.std_logic_1164.all;

entity t_ff is
    port (
        clk : in std_logic;  -- Clock input (positive-edge triggered)
        rst : in std_logic;  -- Asynchronous active-high reset
        T   : in std_logic;  -- Toggle input
        Q   : out std_logic  -- Output of the T Flip-Flop
    );
end entity t_ff;

architecture rtl of t_ff is
    -- Internal signal to hold the current state of Q
    signal Q_internal : std_logic := '0';
begin

    -- Process sensitive to clock and reset signals
    process (clk, rst)
    begin
        if rst = '1' then
            -- Asynchronous reset: Q_internal is cleared to '0'
            Q_internal <= '0';
        elsif rising_edge(clk) then
            -- Synchronous operation on positive clock edge
            if T = '1' then
                -- If T is '1', toggle Q_internal
                Q_internal <= not Q_internal;
            -- If T is '0', Q_internal retains its current value
            end if;
        end if;
    end process;

    -- Assign the internal Q to the output port Q
    Q <= Q_internal;

end architecture rtl;

Writing t_ff.vhdl


## 2. JK Flip-Flop

The JK flip-flop is considered a universal flip-flop because it can be configured to perform the functions of SR, D, and T flip-flops. Its behavior is defined by its J and K inputs:
* J=0, K=0: No change
* J=0, K=1: Reset (Q becomes 0)
* J=1, K=0: Set (Q becomes 1)
* J=1, K=1: Toggle (Q flips)

### Verilog RTL for JK Flip-Flop


In [ ]:
%%writefile jk_ff.v
// Module: jk_ff
// Description: JK Flip-Flop with asynchronous active-high reset.

module jk_ff (
    input clk,  // Clock input (positive-edge triggered)
    input rst,  // Asynchronous active-high reset
    input J,    // J input
    input K,    // K input
    output reg Q // Output of the JK Flip-Flop
);

    // Always block sensitive to positive clock edge or positive reset edge
    always @(posedge clk or posedge rst) begin
        if (rst) begin
            // Asynchronous reset: Q is cleared to 0
            Q <= 1'b0;
        end else begin
            // Synchronous operation:
            case ({J, K}) // Concatenate J and K for case statement
                2'b00: begin // J=0, K=0: No change
                    Q <= Q; // Explicitly state no change
                end
                2'b01: begin // J=0, K=1: Reset Q to 0
                    Q <= 1'b0;
                end
                2'b10: begin // J=1, K=0: Set Q to 1
                    Q <= 1'b1;
                end
                2'b11: begin // J=1, K=1: Toggle Q
                    Q <= ~Q;
                end
            endcase
        end
    end

endmodule

Writing jk_ff.v


### VHDL RTL for JK Flip-Flop


In [ ]:
%%writefile jk_ff.vhdl
-- Entity: jk_ff
-- Description: JK Flip-Flop with asynchronous active-high reset.

library ieee;
use ieee.std_logic_1164.all;

entity jk_ff is
    port (
        clk : in std_logic;  -- Clock input (positive-edge triggered)
        rst : in std_logic;  -- Asynchronous active-high reset
        J   : in std_logic;  -- J input
        K   : in std_logic;  -- K input
        Q   : out std_logic  -- Output of the JK Flip-Flop
    );
end entity jk_ff;

architecture rtl of jk_ff is
    -- Internal signal to hold the current state of Q
    signal Q_internal : std_logic := '0';
begin

    -- Process sensitive to clock and reset signals
    process (clk, rst)
    begin
        if rst = '1' then
            -- Asynchronous reset: Q_internal is cleared to '0'
            Q_internal <= '0';
        elsif rising_edge(clk) then
            -- Synchronous operation on positive clock edge
            if J = '0' and K = '0' then
                -- J=0, K=0: No change
                Q_internal <= Q_internal;
            elsif J = '0' and K = '1' then
                -- J=0, K=1: Reset Q_internal to '0'
                Q_internal <= '0';
            elsif J = '1' and K = '0' then
                -- J=1, K=0: Set Q_internal to '1'
                Q_internal <= '1';
            elsif J = '1' and K = '1' then
                -- J=1, K=1: Toggle Q_internal
                Q_internal <= not Q_internal;
            end if;
        end if;
    end process;

    -- Assign the internal Q to the output port Q
    Q <= Q_internal;

end architecture rtl;

Writing jk_ff.vhdl


## 3. D Flip-Flop

The D (Data or Delay) flip-flop is the simplest type of flip-flop. It captures the value present at its D input on the active clock edge and transfers it to its Q output. It is widely used for creating registers and memories.

### Verilog RTL for D Flip-Flop


In [ ]:
%%writefile d_ff.v
// Module: d_ff
// Description: D (Data) Flip-Flop with asynchronous active-high reset.

module d_ff (
    input clk,  // Clock input (positive-edge triggered)
    input rst,  // Asynchronous active-high reset
    input D,    // Data input
    output reg Q // Output of the D Flip-Flop
);

    // Always block sensitive to positive clock edge or positive reset edge
    always @(posedge clk or posedge rst) begin
        if (rst) begin
            // Asynchronous reset: Q is cleared to 0
            Q <= 1'b0;
        end else begin
            // Synchronous operation: Q takes the value of D on clock edge
            Q <= D;
        end
    end

endmodule

Writing d_ff.v


### VHDL RTL for D Flip-Flop


In [ ]:
%%writefile d_ff.vhdl
-- Entity: d_ff
-- Description: D (Data) Flip-Flop with asynchronous active-high reset.

library ieee;
use ieee.std_logic_1164.all;

entity d_ff is
    port (
        clk : in std_logic;  -- Clock input (positive-edge triggered)
        rst : in std_logic;  -- Asynchronous active-high reset
        D   : in std_logic;  -- Data input
        Q   : out std_logic  -- Output of the D Flip-Flop
    );
end entity d_ff;

architecture rtl of d_ff is
    -- Internal signal to hold the current state of Q
    signal Q_internal : std_logic := '0';
begin

    -- Process sensitive to clock and reset signals
    process (clk, rst)
    begin
        if rst = '1' then
            -- Asynchronous reset: Q_internal is cleared to '0'
            Q_internal <= '0';
        elsif rising_edge(clk) then
            -- Synchronous operation on positive clock edge: Q_internal takes the value of D
            Q_internal <= D;
        end if;
    end process;

    -- Assign the internal Q to the output port Q
    Q <= Q_internal;

end architecture rtl;

Writing d_ff.vhdl


## 4. Clock Divider

A clock divider generates a new clock signal with a lower frequency than the input clock. A common application is to derive slower clocks for various sub-systems from a single high-frequency master clock. Here, we implement a divide-by-2 clock divider, which effectively halves the input clock frequency.

### Verilog RTL for Clock Divider (Divide-by-2)


In [ ]:
%%writefile clk_divider.v
// Module: clk_divider_div2
// Description: Divides the input clock frequency by 2.
//              Generates a clock with half the frequency of 'clk'.

module clk_divider_div2 (
    input clk,     // Input clock
    input rst,     // Asynchronous active-high reset
    output reg clk_div2 // Output clock (half frequency)
);

    // Always block sensitive to positive clock edge or positive reset edge
    always @(posedge clk or posedge rst) begin
        if (rst) begin
            // Asynchronous reset: Output clock is cleared to 0
            clk_div2 <= 1'b0;
        end else begin
            // Synchronous operation: Toggle the output clock on each rising edge of input clock
            clk_div2 <= ~clk_div2;
        end
    end

endmodule

Writing clk_divider.v


### VHDL RTL for Clock Divider (Divide-by-2)


In [ ]:
%%writefile clk_divider.vhdl
-- Entity: clk_divider_div2
-- Description: Divides the input clock frequency by 2.
--              Generates a clock with half the frequency of 'clk'.

library ieee;
use ieee.std_logic_1164.all;

entity clk_divider_div2 is
    port (
        clk      : in  std_logic;  -- Input clock
        rst      : in  std_logic;  -- Asynchronous active-high reset
        clk_div2 : out std_logic   -- Output clock (half frequency)
    );
end entity clk_divider_div2;

architecture rtl of clk_divider_div2 is
    -- Internal signal to hold the state of the divided clock
    signal clk_div2_internal : std_logic := '0';
begin

    -- Process sensitive to clock and reset signals
    process (clk, rst)
    begin
        if rst = '1' then
            -- Asynchronous reset: clk_div2_internal is cleared to '0'
            clk_div2_internal <= '0';
        elsif rising_edge(clk) then
            -- Synchronous operation: Toggle the internal signal on each rising edge of the input clock
            clk_div2_internal <= not clk_div2_internal;
        end if;
    end process;

    -- Assign the internal signal to the output port
    clk_div2 <= clk_div2_internal;

end architecture rtl;

Writing clk_divider.vhdl


## 5. 2-bit Synchronous Counter

A synchronous counter is a type of counter where all flip-flops are clocked simultaneously by the same clock signal. This ensures that all bits of the counter change state at the same time, avoiding propagation delays seen in asynchronous (ripple) counters. This example implements a simple 2-bit up counter that counts from 00 to 11 and then rolls over to 00.

### Verilog RTL for 2-bit Synchronous Counter


In [ ]:
%%writefile sync_counter_2bit.v
// Module: sync_counter_2bit
// Description: A 2-bit synchronous up counter with asynchronous active-high reset.
//              Counts from 00 -> 01 -> 10 -> 11 -> 00...

module sync_counter_2bit (
    input clk,          // Clock input (positive-edge triggered)
    input rst,          // Asynchronous active-high reset
    output reg [1:0] Q  // 2-bit output of the counter
);

    // Always block sensitive to positive clock edge or positive reset edge
    always @(posedge clk or posedge rst) begin
        if (rst) begin
            // Asynchronous reset: Q is cleared to 00
            Q <= 2'b00;
        end else begin
            // Synchronous operation: Increment the counter on each clock edge
            Q <= Q + 1; // Standard increment for binary counter
        end
    end

endmodule

Writing sync_counter_2bit.v


### VHDL RTL for 2-bit Synchronous Counter


In [ ]:
%%writefile sync_counter_2bit.vhdl
-- Entity: sync_counter_2bit
-- Description: A 2-bit synchronous up counter with asynchronous active-high reset.
--              Counts from 00 -> 01 -> 10 -> 11 -> 00...

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all; -- Required for unsigned arithmetic

entity sync_counter_2bit is
    port (
        clk : in std_logic;       -- Clock input (positive-edge triggered)
        rst : in std_logic;       -- Asynchronous active-high reset
        Q   : out std_logic_vector(1 downto 0) -- 2-bit output of the counter
    );
end entity sync_counter_2bit;

architecture rtl of sync_counter_2bit is
    -- Internal signal to hold the current count value.
    -- Using unsigned type for arithmetic operations.
    signal Q_internal : unsigned(1 downto 0) := "00";
begin

    -- Process sensitive to clock and reset signals
    process (clk, rst)
    begin
        if rst = '1' then
            -- Asynchronous reset: Q_internal is cleared to "00"
            Q_internal <= "00";
        elsif rising_edge(clk) then
            -- Synchronous operation: Increment the counter on positive clock edge
            Q_internal <= Q_internal + 1; -- Increment
        end if;
    end process;

    -- Assign the internal unsigned count to the std_logic_vector output port
    Q <= std_logic_vector(Q_internal);

end architecture rtl;

Writing sync_counter_2bit.vhdl


# Testbench Development for Sequential Logic Circuits in Verilog and VHDL

This chapter focuses on creating effective testbenches for the sequential logic circuits introduced in the previous chapter. A testbench is a crucial component of the hardware design flow, allowing designers to verify the functional correctness of their RTL (Register Transfer Level) code before proceeding to synthesis and physical implementation. In a Google Colab environment, testbenches are particularly useful for quick simulation and debugging.

## The Role of a Testbench

A testbench is a separate HDL module that instantiates the Device Under Test (DUT) – the circuit you want to verify. It does not have any physical hardware equivalent; its sole purpose is to:

1.  **Generate Stimulus:** Provide appropriate input signals (like clock, reset, data inputs) to the DUT.
2.  **Monitor Outputs:** Observe the DUT's output signals.
3.  **Verify Behavior:** Compare observed outputs against expected behavior, often through assertions or simple checks.
4.  **Capture Waveforms:** Generate waveform dump files (e.g., VCD files) for visual analysis using waveform viewers (like GTKWave).

## Common Testbench Elements

For synchronous sequential circuits, a testbench typically includes:

* **Clock Generation:** A continuous oscillating signal to drive the DUT's clock input.
* **Reset Generation:** A pulse to bring the DUT to a known initial state.
* **Input Stimulus:** Varying input signals over time to test different operating conditions.
* **Instantiation of DUT:** Connecting the testbench's signals to the DUT's ports.
* **Simulation Control:** Directives to start and stop the simulation, and to enable waveform dumping.

## Testbench Implementations for Sequential Circuits

We will now generate testbenches for the T, JK, and D Flip-Flops, the Clock Divider, and the 2-bit Synchronous Counter, in both Verilog and VHDL.

---

## 1. T Flip-Flop Testbench

This testbench will apply various `T` input values, observe `Q`, and demonstrate the toggle behavior.

### Verilog Testbench for T Flip-Flop


In [ ]:
%%writefile t_ff_tb.v
// Testbench for T Flip-Flop (t_ff.v)

`timescale 1ns / 1ps // Define timescale for simulation

module t_ff_tb;

    // Declare testbench signals for DUT ports
    reg clk;
    reg rst;
    reg T;
    wire Q;

    // Instantiate the Device Under Test (DUT)
    // Connect testbench signals to DUT ports by name
    t_ff dut (
        .clk(clk),
        .rst(rst),
        .T(T),
        .Q(Q)
    );

    // Clock generation
    // Initial clock value and continuous toggling
    initial begin
        clk = 0; // Initialize clock to 0
        forever #5 clk = ~clk; // Toggle clock every 5 time units (10ns period)
    end

    // Test stimulus
    initial begin
        // Dump waves to wave.vcd file for waveform viewing
        $dumpfile("wave.vcd");
        $dumpvars(0, t_ff_tb); // Dump all signals in the current scope (t_ff_tb)

        // 1. Initial Reset
        rst = 1; // Assert reset
        T   = 0; // T input doesn't matter during reset
        #10;     // Hold reset for 10ns
        rst = 0; // De-assert reset
        #10;     // Wait for some time after reset

        // 2. Test T = 0 (No change)
        T = 0;
        #20; // Allow 2 clock cycles (2 * 10ns = 20ns)
             // Q should remain 0

        // 3. Test T = 1 (Toggle)
        T = 1;
        #20; // Q should toggle 2 times (0 -> 1 -> 0)

        // 4. Test T = 0 again (No change)
        T = 0;
        #20; // Q should remain as its last toggled value

        // 5. Test T = 1 multiple toggles
        T = 1;
        #50; // Q should toggle multiple times

        // 6. Assert reset again to observe reset behavior
        rst = 1;
        #10;
        rst = 0;
        #10; // Q should go to 0 on reset, then stay 0 if T=0

        // 7. Test T=1 after reset
        T = 1;
        #30;

        $finish; // End simulation
    end

    // Monitor outputs (optional, but good for quick checks in console)
    // always @(posedge clk) begin
    //     $display("Time: %0t, T: %b, Q: %b", $time, T, Q);
    // end

endmodule

Writing t_ff_tb.v


### VHDL Testbench for T Flip-Flop


In [ ]:
%%writefile t_ff_tb.vhdl
-- Testbench for T Flip-Flop (t_ff.vhdl)

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all; -- Required for GHDL simulation control

entity t_ff_tb is
end entity t_ff_tb;

architecture tb of t_ff_tb is

    -- Component declaration for the DUT
    component t_ff is
        port (
            clk : in std_logic;
            rst : in std_logic;
            T   : in std_logic;
            Q   : out std_logic
        );
    end component t_ff;

    -- Declare signals for testbench
    signal clk_tb : std_logic := '0';
    signal rst_tb : std_logic := '0';
    signal T_tb   : std_logic := '0';
    signal Q_tb   : std_logic;

    -- Clock period definition
    constant CLK_PERIOD : time := 10 ns;

begin

    -- Instantiate the DUT
    dut_inst : t_ff
        port map (
            clk => clk_tb,
            rst => rst_tb,
            T   => T_tb,
            Q   => Q_tb
        );

    -- Clock generation process
    clk_gen_proc : process
    begin
        loop
            clk_tb <= '0';
            wait for CLK_PERIOD / 2;
            clk_tb <= '1';
            wait for CLK_PERIOD / 2;
        end loop;
    end process clk_gen_proc;

    -- Test stimulus process
    stim_proc : process
    begin
        -- 1. Initial Reset
        rst_tb <= '1'; -- Assert reset
        T_tb   <= '0'; -- T input doesn't matter during reset
        wait for CLK_PERIOD; -- Hold reset for one clock period
        rst_tb <= '0'; -- De-assert reset
        wait for CLK_PERIOD; -- Wait after reset

        -- 2. Test T = '0' (No change)
        T_tb <= '0';
        wait for 2 * CLK_PERIOD; -- Allow 2 clock cycles
                                 -- Q_tb should remain '0'

        -- 3. Test T = '1' (Toggle)
        T_tb <= '1';
        wait for 2 * CLK_PERIOD; -- Q_tb should toggle 2 times ('0' -> '1' -> '0')

        -- 4. Test T = '0' again (No change)
        T_tb <= '0';
        wait for 2 * CLK_PERIOD; -- Q_tb should remain as its last toggled value

        -- 5. Test T = '1' multiple toggles
        T_tb <= '1';
        wait for 5 * CLK_PERIOD; -- Q_tb should toggle multiple times

        -- 6. Assert reset again to observe reset behavior
        rst_tb <= '1';
        wait for CLK_PERIOD;
        rst_tb <= '0';
        wait for CLK_PERIOD; -- Q_tb should go to '0' on reset

        -- 7. Test T='1' after reset
        T_tb <= '1';
        wait for 3 * CLK_PERIOD;

        wait; -- End of simulation (equivalent to $finish in Verilog)
    end process stim_proc;

end architecture tb;

Writing t_ff_tb.vhdl


---

## 2. JK Flip-Flop Testbench

This testbench will apply various combinations of `J` and `K` inputs to verify all four operational modes (No Change, Reset, Set, Toggle).

### Verilog Testbench for JK Flip-Flop


In [ ]:
%%writefile jk_ff_tb.v
// Testbench for JK Flip-Flop (jk_ff.v)

`timescale 1ns / 1ps

module jk_ff_tb;

    // Declare testbench signals
    reg clk;
    reg rst;
    reg J;
    reg K;
    wire Q;

    // Instantiate the DUT
    jk_ff dut (
        .clk(clk),
        .rst(rst),
        .J(J),
        .K(K),
        .Q(Q)
    );

    // Clock generation
    initial begin
        clk = 0;
        forever #5 clk = ~clk; // 10ns period
    end

    // Test stimulus
    initial begin
        $dumpfile("wave.vcd");
        $dumpvars(0, jk_ff_tb);

        // 1. Initial Reset
        rst = 1;
        J = 0; K = 0;
        #10;
        rst = 0;
        #10; // Q should be 0

        // 2. Test J=0, K=0 (No change)
        J = 0; K = 0;
        #20; // Q should remain 0

        // 3. Test J=1, K=0 (Set Q to 1)
        J = 1; K = 0;
        #20; // Q should become 1

        // 4. Test J=0, K=0 (No change, Q should remain 1)
        J = 0; K = 0;
        #20; // Q should remain 1

        // 5. Test J=0, K=1 (Reset Q to 0)
        J = 0; K = 1;
        #20; // Q should become 0

        // 6. Test J=1, K=1 (Toggle, Q should become 1)
        J = 1; K = 1;
        #20; // Q should toggle (0 -> 1)

        // 7. Test J=1, K=1 again (Toggle, Q should become 0)
        J = 1; K = 1;
        #20; // Q should toggle (1 -> 0)

        // 8. Test J=1, K=0 (Set Q to 1)
        J = 1; K = 0;
        #20; // Q should become 1

        // 9. Assert reset again
        rst = 1;
        #10;
        rst = 0;
        #10; // Q should be 0

        $finish;
    end

endmodule

Writing jk_ff_tb.v


### VHDL Testbench for JK Flip-Flop


In [ ]:
%%writefile jk_ff_tb.vhdl
-- Testbench for JK Flip-Flop (jk_ff.vhdl)

library ieee;
use ieee.std_logic_1164.all;

entity jk_ff_tb is
end entity jk_ff_tb;

architecture tb of jk_ff_tb is

    component jk_ff is
        port (
            clk : in std_logic;
            rst : in std_logic;
            J   : in std_logic;
            K   : in std_logic;
            Q   : out std_logic
        );
    end component jk_ff;

    signal clk_tb : std_logic := '0';
    signal rst_tb : std_logic := '0';
    signal J_tb   : std_logic := '0';
    signal K_tb   : std_logic := '0';
    signal Q_tb   : std_logic;

    constant CLK_PERIOD : time := 10 ns;

begin

    dut_inst : jk_ff
        port map (
            clk => clk_tb,
            rst => rst_tb,
            J   => J_tb,
            K   => K_tb,
            Q   => Q_tb
        );

    clk_gen_proc : process
    begin
        loop
            clk_tb <= '0';
            wait for CLK_PERIOD / 2;
            clk_tb <= '1';
            wait for CLK_PERIOD / 2;
        end loop;
    end process clk_gen_proc;

    stim_proc : process
    begin
        -- 1. Initial Reset
        rst_tb <= '1';
        J_tb   <= '0'; K_tb <= '0';
        wait for CLK_PERIOD;
        rst_tb <= '0';
        wait for CLK_PERIOD; -- Q_tb should be '0'

        -- 2. Test J='0', K='0' (No change)
        J_tb <= '0'; K_tb <= '0';
        wait for 2 * CLK_PERIOD; -- Q_tb should remain '0'

        -- 3. Test J='1', K='0' (Set Q to '1')
        J_tb <= '1'; K_tb <= '0';
        wait for 2 * CLK_PERIOD; -- Q_tb should become '1'

        -- 4. Test J='0', K='0' (No change, Q should remain '1')
        J_tb <= '0'; K_tb <= '0';
        wait for 2 * CLK_PERIOD; -- Q_tb should remain '1'

        -- 5. Test J='0', K='1' (Reset Q to '0')
        J_tb <= '0'; K_tb <= '1';
        wait for 2 * CLK_PERIOD; -- Q_tb should become '0'

        -- 6. Test J='1', K='1' (Toggle, Q should become '1')
        J_tb <= '1'; K_tb <= '1';
        wait for 2 * CLK_PERIOD; -- Q_tb should toggle ('0' -> '1')

        -- 7. Test J='1', K='1' again (Toggle, Q should become '0')
        J_tb <= '1'; K_tb <= '1';
        wait for 2 * CLK_PERIOD; -- Q_tb should toggle ('1' -> '0')

        -- 8. Test J='1', K='0' (Set Q to '1')
        J_tb <= '1'; K_tb <= '0';
        wait for 2 * CLK_PERIOD; -- Q_tb should become '1'

        -- 9. Assert reset again
        rst_tb <= '1';
        wait for CLK_PERIOD;
        rst_tb <= '0';
        wait for CLK_PERIOD; -- Q_tb should be '0'

        wait;
    end process stim_proc;

end architecture tb;

Writing jk_ff_tb.vhdl


---

## 3. D Flip-Flop Testbench

This testbench will provide various `D` input values and verify that `Q` captures `D` on the clock edge.

### Verilog Testbench for D Flip-Flop


In [ ]:
%%writefile d_ff_tb.v
// Testbench for D Flip-Flop (d_ff.v)

`timescale 1ns / 1ps

module d_ff_tb;

    // Declare testbench signals
    reg clk;
    reg rst;
    reg D;
    wire Q;

    // Instantiate the DUT
    d_ff dut (
        .clk(clk),
        .rst(rst),
        .D(D),
        .Q(Q)
    );

    // Clock generation
    initial begin
        clk = 0;
        forever #5 clk = ~clk; // 10ns period
    end

    // Test stimulus
    initial begin
        $dumpfile("wave.vcd");
        $dumpvars(0, d_ff_tb);

        // 1. Initial Reset
        rst = 1;
        D   = 0;
        #10;
        rst = 0;
        #10; // Q should be 0

        // 2. Test D = 1
        D = 1;
        #10; // Q should become 1 on the next posedge clk

        // 3. Test D = 0
        D = 0;
        #10; // Q should become 0 on the next posedge clk

        // 4. Test D = 1
        D = 1;
        #20; // Q should become 1, then stay 1

        // 5. Change D between clock edges
        D = 0; // Set D to 0
        #3;    // Wait before clock edge (Q should still be 1)
        D = 1; // Change D to 1 before clock edge (Q should capture this D=1)
        #10;   // Q should become 1

        // 6. Change D after clock edge
        D = 0; // Set D to 0
        #7;    // Wait after clock edge (Q should still be 1)
        D = 1; // Change D to 1 (Q should still be 1 until next clock edge)
        #10;   // Q should become 1

        // 7. Assert reset again
        rst = 1;
        #10;
        rst = 0;
        #10; // Q should be 0

        $finish;
    end

endmodule

Writing d_ff_tb.v


### VHDL Testbench for D Flip-Flop


In [ ]:
%%writefile d_ff_tb.vhdl
-- Testbench for D Flip-Flop (d_ff.vhdl)

library ieee;
use ieee.std_logic_1164.all;

entity d_ff_tb is
end entity d_ff_tb;

architecture tb of d_ff_tb is

    component d_ff is
        port (
            clk : in std_logic;
            rst : in std_logic;
            D   : in std_logic;
            Q   : out std_logic
        );
    end component d_ff;

    signal clk_tb : std_logic := '0';
    signal rst_tb : std_logic := '0';
    signal D_tb   : std_logic := '0';
    signal Q_tb   : std_logic;

    constant CLK_PERIOD : time := 10 ns;

begin

    dut_inst : d_ff
        port map (
            clk => clk_tb,
            rst => rst_tb,
            D   => D_tb,
            Q   => Q_tb
        );

    clk_gen_proc : process
    begin
        loop
            clk_tb <= '0';
            wait for CLK_PERIOD / 2;
            clk_tb <= '1';
            wait for CLK_PERIOD / 2;
        end loop;
    end process clk_gen_proc;

    stim_proc : process
    begin
        -- 1. Initial Reset
        rst_tb <= '1';
        D_tb   <= '0';
        wait for CLK_PERIOD;
        rst_tb <= '0';
        wait for CLK_PERIOD; -- Q_tb should be '0'

        -- 2. Test D = '1'
        D_tb <= '1';
        wait for CLK_PERIOD; -- Q_tb should become '1' on the next rising_edge(clk)

        -- 3. Test D = '0'
        D_tb <= '0';
        wait for CLK_PERIOD; -- Q_tb should become '0' on the next rising_edge(clk)

        -- 4. Test D = '1'
        D_tb <= '1';
        wait for 2 * CLK_PERIOD; -- Q_tb should become '1', then stay '1'

        -- 5. Change D between clock edges
        D_tb <= '0'; -- Set D to '0'
        wait for CLK_PERIOD / 2 - 2 ns; -- Wait before clock edge (Q_tb should still be '1')
        D_tb <= '1'; -- Change D to '1' before clock edge (Q_tb should capture this D='1')
        wait for CLK_PERIOD / 2 + 2 ns; -- Q_tb should become '1'

        -- 6. Change D after clock edge
        D_tb <= '0'; -- Set D to '0'
        wait for CLK_PERIOD / 2 + 2 ns; -- Wait after clock edge (Q_tb should still be '1')
        D_tb <= '1'; -- Change D to '1' (Q_tb should still be '1' until next clock edge)
        wait for CLK_PERIOD / 2 - 2 ns; -- Q_tb should become '1'

        -- 7. Assert reset again
        rst_tb <= '1';
        wait for CLK_PERIOD;
        rst_tb <= '0';
        wait for CLK_PERIOD; -- Q_tb should be '0'

        wait;
    end process stim_proc;

end architecture tb;

Writing d_ff_tb.vhdl


---

## 4. Clock Divider Testbench

This testbench will observe the output of the clock divider, confirming its frequency is half of the input clock.

### Verilog Testbench for Clock Divider


In [ ]:
%%writefile clk_divider_tb.v
// Testbench for Clock Divider (clk_divider.v)

`timescale 1ns / 1ps

module clk_divider_div2_tb;

    // Declare testbench signals
    reg clk;
    reg rst;
    wire clk_div2;

    // Instantiate the DUT
    clk_divider_div2 dut (
        .clk(clk),
        .rst(rst),
        .clk_div2(clk_div2)
    );

    // Clock generation
    initial begin
        clk = 0;
        forever #5 clk = ~clk; // 10ns period (5ns high, 5ns low)
    end

    // Test stimulus
    initial begin
        $dumpfile("wave.vcd");
        $dumpvars(0, clk_divider_div2_tb);

        // 1. Initial Reset
        rst = 1;
        #10;
        rst = 0;
        #10; // clk_div2 should be 0

        // 2. Observe divided clock for several cycles
        // The clk_div2 should toggle every 10ns, resulting in a 20ns period.
        #100; // Run for 100ns (10 input clock cycles, 5 output clock cycles)

        // 3. Assert reset again during operation
        rst = 1;
        #10;
        rst = 0;
        #10;

        // 4. Continue observing
        #50;

        $finish;
    end

    // Optional: Display timing info
    // always @(posedge clk) begin
    //     $display("Time: %0t, clk: %b, clk_div2: %b", $time, clk, clk_div2);
    // end

endmodule

Writing clk_divider_tb.v


### VHDL Testbench for Clock Divider


In [ ]:
%%writefile clk_divider_tb.vhdl
-- Testbench for Clock Divider (clk_divider.vhdl)

library ieee;
use ieee.std_logic_1164.all;

entity clk_divider_div2_tb is
end entity clk_divider_div2_tb;

architecture tb of clk_divider_div2_tb is

    component clk_divider_div2 is
        port (
            clk      : in  std_logic;
            rst      : in  std_logic;
            clk_div2 : out std_logic
        );
    end component clk_divider_div2;

    signal clk_tb      : std_logic := '0';
    signal rst_tb      : std_logic := '0';
    signal clk_div2_tb : std_logic;

    constant CLK_PERIOD : time := 10 ns;

begin

    dut_inst : clk_divider_div2
        port map (
            clk      => clk_tb,
            rst      => rst_tb,
            clk_div2 => clk_div2_tb
        );

    clk_gen_proc : process
    begin
        loop
            clk_tb <= '0';
            wait for CLK_PERIOD / 2;
            clk_tb <= '1';
            wait for CLK_PERIOD / 2;
        end loop;
    end process clk_gen_proc;

    stim_proc : process
    begin
        -- 1. Initial Reset
        rst_tb <= '1';
        wait for CLK_PERIOD;
        rst_tb <= '0';
        wait for CLK_PERIOD; -- clk_div2_tb should be '0'

        -- 2. Observe divided clock for several cycles
        -- The clk_div2_tb should toggle every 10ns, resulting in a 20ns period.
        wait for 10 * CLK_PERIOD; -- Run for 10 input clock cycles

        -- 3. Assert reset again during operation
        rst_tb <= '1';
        wait for CLK_PERIOD;
        rst_tb <= '0';
        wait for CLK_PERIOD;

        -- 4. Continue observing
        wait for 5 * CLK_PERIOD;

        wait;
    end process stim_proc;

end architecture tb;

Writing clk_divider_tb.vhdl


---

## 5. 2-bit Synchronous Counter Testbench

This testbench will verify that the counter increments correctly from `00` to `11` and then wraps around.

### Verilog Testbench for 2-bit Synchronous Counter


In [ ]:
%%writefile sync_counter_2bit_tb.v
// Testbench for 2-bit Synchronous Counter (sync_counter_2bit.v)

`timescale 1ns / 1ps

module sync_counter_2bit_tb;

    // Declare testbench signals
    reg clk;
    reg rst;
    wire [1:0] Q;

    // Instantiate the DUT
    sync_counter_2bit dut (
        .clk(clk),
        .rst(rst),
        .Q(Q)
    );

    // Clock generation
    initial begin
        clk = 0;
        forever #5 clk = ~clk; // 10ns period
    end

    // Test stimulus
    initial begin
        $dumpfile("wave.vcd");
        $dumpvars(0, sync_counter_2bit_tb);

        // 1. Initial Reset
        rst = 1;
        #10;
        rst = 0;
        #10; // Q should be 00

        // 2. Observe counting sequence (00 -> 01 -> 10 -> 11 -> 00)
        #50; // Run for 5 clock cycles to see wraps around

        // 3. Assert reset again during counting
        rst = 1;
        #10;
        rst = 0;
        #10; // Q should go to 00

        // 4. Continue counting after reset
        #30;

        $finish;
    end

    // Optional: Monitor outputs
    // always @(posedge clk) begin
    //     $display("Time: %0t, Q: %b", $time, Q);
    // end

endmodule

Writing sync_counter_2bit_tb.v


### VHDL Testbench for 2-bit Synchronous Counter


In [ ]:
%%writefile sync_counter_2bit_tb.vhdl
-- Testbench for 2-bit Synchronous Counter (sync_counter_2bit.vhdl)

library ieee;
use ieee.std_logic_1164.all;
use ieee.numeric_std.all; -- For comparing unsigned values in simulation

entity sync_counter_2bit_tb is
end entity sync_counter_2bit_tb;

architecture tb of sync_counter_2bit_tb is

    component sync_counter_2bit is
        port (
            clk : in std_logic;
            rst : in std_logic;
            Q   : out std_logic_vector(1 downto 0)
        );
    end component sync_counter_2bit;

    signal clk_tb : std_logic := '0';
    signal rst_tb : std_logic := '0';
    signal Q_tb   : std_logic_vector(1 downto 0);

    constant CLK_PERIOD : time := 10 ns;

begin

    dut_inst : sync_counter_2bit
        port map (
            clk => clk_tb,
            rst => rst_tb,
            Q   => Q_tb
        );

    clk_gen_proc : process
    begin
        loop
            clk_tb <= '0';
            wait for CLK_PERIOD / 2;
            clk_tb <= '1';
            wait for CLK_PERIOD / 2;
        end loop;
    end process clk_gen_proc;

    stim_proc : process
    begin
        -- 1. Initial Reset
        rst_tb <= '1';
        wait for CLK_PERIOD;
        rst_tb <= '0';
        wait for CLK_PERIOD; -- Q_tb should be "00"

        -- 2. Observe counting sequence ("00" -> "01" -> "10" -> "11" -> "00")
        wait for 5 * CLK_PERIOD; -- Run for 5 clock cycles to see wrap around

        -- 3. Assert reset again during counting
        rst_tb <= '1';
        wait for CLK_PERIOD;
        rst_tb <= '0';
        wait for CLK_PERIOD; -- Q_tb should go to "00"

        -- 4. Continue counting after reset
        wait for 3 * CLK_PERIOD;

        wait;
    end process stim_proc;

end architecture tb;

Writing sync_counter_2bit_tb.vhdl


---

## Running Simulations in Colab

After creating these `.v` and `.vhdl` files using `%%writefile` cells, you can compile and simulate them in subsequent `%%bash` cells within your Colab notebook.

### For Verilog (using Icarus Verilog):

```bash
# Compile
iverilog -o t_ff_sim t_ff.v t_ff_tb.v

# Run simulation (this will generate wave.vcd)
./t_ff_sim
```

### For VHDL (using GHDL):

```bash
# Analyze (compile) the design unit
ghdl -a t_ff.vhdl t_ff_tb.vhdl

# Elaborate (link) the design
ghdl -e t_ff_tb

# Run simulation (this will also generate a VCD file named dump.vcd by default, or specified by --wave)
ghdl -r t_ff_tb --wave=wave.ghdl.vcd --stop-time=100ns
```

Remember to replace `t_ff` with the appropriate module/entity name for each circuit when running the commands. After simulation, you can download the generated `wave.vcd` (or `wave.ghdl.vcd`) file from your Colab environment and view it using a waveform viewer like GTKWave (which you would run locally on your machine).

# Simulating Digital Logic Designs in Google Colab

This chapter provides a practical guide to simulating your Verilog and VHDL designs within the Google Colab environment using the previously installed Icarus Verilog and GHDL tools. Simulation is a critical step in the digital design flow, allowing you to verify the functional correctness of your hardware description language (HDL) code.

## The Simulation Process

Simulating an HDL design typically involves two main steps:

1.  **Compilation/Analysis & Elaboration:** The HDL code (RTL and testbench) is processed by the simulator. This phase checks for syntax errors, creates an internal representation of the design, and resolves all references. For VHDL, this is often a two-step process: `analyze` for individual files and `elaborate` for linking them into a complete design.
2.  **Simulation Execution:** The elaborated design is then executed over time, based on the stimulus provided by the testbench. During this phase, signal values change, and these changes can be recorded in a waveform dump file (e.g., VCD - Value Change Dump) for visual inspection.

## Verilog Simulation with Icarus Verilog (`iverilog`)

Icarus Verilog is a free and open-source Verilog compiler and simulator. It compiles Verilog source code into a format that can then be executed by its simulation engine.

### `%%bash` Script for Verilog Simulation

The following script demonstrates how to compile and run a Verilog testbench, generate a VCD file, and capture simulation logs.


In [ ]:
%%writefile clk_divider_div2.vhdl
-- Entity: clk_divider_div2
-- Description: Divides the input clock frequency by 2.
--              Generates a clock with half the frequency of 'clk'.

library ieee;
use ieee.std_logic_1164.all;

entity clk_divider_div2 is
    port (
        clk      : in  std_logic;   -- Input clock
        rst      : in  std_logic;   -- Asynchronous active-high reset
        clk_div2 : out std_logic    -- Output clock (half frequency)
    );
end entity clk_divider_div2;

architecture rtl of clk_divider_div2 is
    -- Internal signal to hold the state of the divided clock
    signal clk_div2_internal : std_logic := '0';
begin

    -- Process sensitive to clock and reset signals
    process (clk, rst)
    begin
        if rst = '1' then
            -- Asynchronous reset: clk_div2_internal is cleared to '0'
            clk_div2_internal <= '0';
        elsif rising_edge(clk) then
            -- Synchronous operation: Toggle the internal signal on each rising edge of the input clock
            clk_div2_internal <= not clk_div2_internal;
        end if;
    end process;

    -- Assign the internal signal to the output port
    clk_div2 <= clk_div2_internal;

end architecture rtl;

Writing clk_divider_div2.vhdl


In [ ]:
%%writefile clk_divider_div2_tb.vhdl
-- Testbench for Clock Divider (clk_divider_div2.vhdl)

library ieee;
use ieee.std_logic_1164.all;

entity clk_divider_div2_tb is
end entity clk_divider_div2_tb;

architecture tb of clk_divider_div2_tb is

    component clk_divider_div2 is
        port (
            clk      : in  std_logic;
            rst      : in  std_logic;
            clk_div2 : out std_logic
        );
    end component clk_divider_div2;

    signal clk_tb      : std_logic := '0';
    signal rst_tb      : std_logic := '0';
    signal clk_div2_tb : std_logic;

    constant CLK_PERIOD : time := 10 ns;

begin

    dut_inst : clk_divider_div2
        port map (
            clk      => clk_tb,
            rst      => rst_tb,
            clk_div2 => clk_div2_tb
        );

    clk_gen_proc : process
    begin
        loop
            clk_tb <= '0';
            wait for CLK_PERIOD / 2;
            clk_tb <= '1';
            wait for CLK_PERIOD / 2;
        end loop;
    end process clk_gen_proc;

    stim_proc : process
    begin
        -- 1. Initial Reset
        rst_tb <= '1';
        wait for CLK_PERIOD;
        rst_tb <= '0';
        wait for CLK_PERIOD;

        -- 2. Observe divided clock for several cycles
        wait for 10 * CLK_PERIOD;

        -- 3. Assert reset again during operation
        rst_tb <= '1';
        wait for CLK_PERIOD;
        rst_tb <= '0';
        wait for CLK_PERIOD;

        -- 4. Continue observing
        wait for 5 * CLK_PERIOD;

        wait;
    end process stim_proc;

end architecture tb;

Overwriting clk_divider_div2_tb.vhdl


In [ ]:
%%bash

# --- Verilog Simulation for T Flip-Flop ---
echo "--- Simulating T Flip-Flop (t_ff.v) ---"

# Define the module name for consistent VCD and log filenames
VERILOG_MODULE_NAME="t_ff"

# Compile the Verilog RTL and testbench files
# -o: specifies the output executable name
# >&: redirects both stdout and stderr to the log file
# 2>&1: redirects stderr to stdout, which is then redirected to the file.
echo "Compiling ${VERILOG_MODULE_NAME}.v and ${VERILOG_MODULE_NAME}_tb.v..."
iverilog -o ${VERILOG_MODULE_NAME}_sim ${VERILOG_MODULE_NAME}.v ${VERILOG_MODULE_NAME}_tb.v > ${VERILOG_MODULE_NAME}_compile.log 2>&1

# Check if compilation was successful
if [ $? -eq 0 ]; then
    echo "Compilation successful. Running simulation..."
    # Run the compiled simulation executable
    # The $dumpfile and $dumpvars commands in the testbench will create wave.vcd
    ./${VERILOG_MODULE_NAME}_sim > ${VERILOG_MODULE_NAME}_sim.log 2>&1

    # Check if simulation ran successfully
    if [ $? -eq 0 ]; then
        echo "Simulation for ${VERILOG_MODULE_NAME} completed successfully."
        echo "Generated ${VERILOG_MODULE_NAME}_sim.log and wave.vcd"
        echo "You can download wave.vcd and view it with GTKWave."
        echo "✅ ${VERILOG_MODULE_NAME} simulation PASSED."
    else
        echo "Simulation for ${VERILOG_MODULE_NAME} FAILED. Check ${VERILOG_MODULE_NAME}_sim.log for details."
        echo "❌ ${VERILOG_MODULE_NAME} simulation FAILED."
    fi
else
    echo "Compilation for ${VERILOG_MODULE_NAME} FAILED. Check ${VERILOG_MODULE_NAME}_compile.log for details."
    echo "❌ ${VERILOG_MODULE_NAME} compilation FAILED."
fi

echo "" # Add a blank line for readability between different module simulations

# --- Verilog Simulation for JK Flip-Flop ---
echo "--- Simulating JK Flip-Flop (jk_ff.v) ---"
VERILOG_MODULE_NAME="jk_ff"
echo "Compiling ${VERILOG_MODULE_NAME}.v and ${VERILOG_MODULE_NAME}_tb.v..."
iverilog -o ${VERILOG_MODULE_NAME}_sim ${VERILOG_MODULE_NAME}.v ${VERILOG_MODULE_NAME}_tb.v > ${VERILOG_MODULE_NAME}_compile.log 2>&1
if [ $? -eq 0 ]; then
    echo "Compilation successful. Running simulation..."
    ./${VERILOG_MODULE_NAME}_sim > ${VERILOG_MODULE_NAME}_sim.log 2>&1
    if [ $? -eq 0 ]; then
        echo "Simulation for ${VERILOG_MODULE_NAME} completed successfully."
        echo "Generated ${VERILOG_MODULE_NAME}_sim.log and wave.vcd"
        echo "✅ ${VERILOG_MODULE_NAME} simulation PASSED."
    else
        echo "Simulation for ${VERILOG_MODULE_NAME} FAILED. Check ${VERILOG_MODULE_NAME}_sim.log for details."
        echo "❌ ${VERILOG_MODULE_NAME} simulation FAILED."
    fi
else
    echo "Compilation for ${VERILOG_MODULE_NAME} FAILED. Check ${VERILOG_MODULE_NAME}_compile.log for details."
    echo "❌ ${VERILOG_MODULE_NAME} compilation FAILED."
fi
echo ""

# --- Verilog Simulation for D Flip-Flop ---
echo "--- Simulating D Flip-Flop (d_ff.v) ---"
VERILOG_MODULE_NAME="d_ff"
echo "Compiling ${VERILOG_MODULE_NAME}.v and ${VERILOG_MODULE_NAME}_tb.v..."
iverilog -o ${VERILOG_MODULE_NAME}_sim ${VERILOG_MODULE_NAME}.v ${VERILOG_MODULE_NAME}_tb.v > ${VERILOG_MODULE_NAME}_compile.log 2>&1
if [ $? -eq 0 ]; then
    echo "Compilation successful. Running simulation..."
    ./${VERILOG_MODULE_NAME}_sim > ${VERILOG_MODULE_NAME}_sim.log 2>&1
    if [ $? -eq 0 ]; then
        echo "Simulation for ${VERILOG_MODULE_NAME} completed successfully."
        echo "Generated ${VERILOG_MODULE_NAME}_sim.log and wave.vcd"
        echo "✅ ${VERILOG_MODULE_NAME} simulation PASSED."
    else
        echo "Simulation for ${VERILOG_MODULE_NAME} FAILED. Check ${VERILOG_MODULE_NAME}_sim.log for details."
        echo "❌ ${VERILOG_MODULE_NAME} simulation FAILED."
    fi
else
    echo "Compilation for ${VERILOG_MODULE_NAME} FAILED. Check ${VERILOG_MODULE_NAME}_compile.log for details."
    echo "❌ ${VERILOG_MODULE_NAME} compilation FAILED."
fi
echo ""

# --- Verilog Simulation for Clock Divider ---
echo "--- Simulating Clock Divider (clk_divider.v) ---"
VERILOG_MODULE_NAME="clk_divider"
echo "Compiling ${VERILOG_MODULE_NAME}.v and ${VERILOG_MODULE_NAME}_tb.v..."
iverilog -o ${VERILOG_MODULE_NAME}_sim ${VERILOG_MODULE_NAME}.v ${VERILOG_MODULE_NAME}_tb.v > ${VERILOG_MODULE_NAME}_compile.log 2>&1
if [ $? -eq 0 ]; then
    echo "Compilation successful. Running simulation..."
    ./${VERILOG_MODULE_NAME}_sim > ${VERILOG_MODULE_NAME}_sim.log 2>&1
    if [ $? -eq 0 ]; then
        echo "Simulation for ${VERILOG_MODULE_NAME} completed successfully."
        echo "Generated ${VERILOG_MODULE_NAME}_sim.log and wave.vcd"
        echo "✅ ${VERILOG_MODULE_NAME} simulation PASSED."
    else
        echo "Simulation for ${VERILOG_MODULE_NAME} FAILED. Check ${VERILOG_MODULE_NAME}_sim.log for details."
        echo "❌ ${VERILOG_MODULE_NAME} simulation FAILED."
    fi
else
    echo "Compilation for ${VERILOG_MODULE_NAME} FAILED. Check ${VERILOG_MODULE_NAME}_compile.log for details."
    echo "❌ ${VERILOG_MODULE_NAME} compilation FAILED."
fi
echo ""

# --- Verilog Simulation for 2-bit Synchronous Counter ---
echo "--- Simulating 2-bit Synchronous Counter (sync_counter_2bit.v) ---"
VERILOG_MODULE_NAME="sync_counter_2bit"
echo "Compiling ${VERILOG_MODULE_NAME}.v and ${VERILOG_MODULE_NAME}_tb.v..."
iverilog -o ${VERILOG_MODULE_NAME}_sim ${VERILOG_MODULE_NAME}.v ${VERILOG_MODULE_NAME}_tb.v > ${VERILOG_MODULE_NAME}_compile.log 2>&1
if [ $? -eq 0 ]; then
    echo "Compilation successful. Running simulation..."
    ./${VERILOG_MODULE_NAME}_sim > ${VERILOG_MODULE_NAME}_sim.log 2>&1
    if [ $? -eq 0 ]; then
        echo "Simulation for ${VERILOG_MODULE_NAME} completed successfully."
        echo "Generated ${VERILOG_MODULE_NAME}_sim.log and wave.vcd"
        echo "✅ ${VERILOG_MODULE_NAME} simulation PASSED."
    else
        echo "Simulation for ${VERILOG_MODULE_NAME} FAILED. Check ${VERILOG_MODULE_NAME}_sim.log for details."
        echo "❌ ${VERILOG_MODULE_NAME} simulation FAILED."
    fi
else
    echo "Compilation for ${VERILOG_MODULE_NAME} FAILED. Check ${VERILOG_MODULE_NAME}_compile.log for details."
    echo "❌ ${VERILOG_MODULE_NAME} compilation FAILED."
fi
echo ""

--- Simulating T Flip-Flop (t_ff.v) ---
Compiling t_ff.v and t_ff_tb.v...
Compilation successful. Running simulation...
Simulation for t_ff completed successfully.
Generated t_ff_sim.log and wave.vcd
You can download wave.vcd and view it with GTKWave.
✅ t_ff simulation PASSED.

--- Simulating JK Flip-Flop (jk_ff.v) ---
Compiling jk_ff.v and jk_ff_tb.v...
Compilation successful. Running simulation...
Simulation for jk_ff completed successfully.
Generated jk_ff_sim.log and wave.vcd
✅ jk_ff simulation PASSED.

--- Simulating D Flip-Flop (d_ff.v) ---
Compiling d_ff.v and d_ff_tb.v...
Compilation successful. Running simulation...
Simulation for d_ff completed successfully.
Generated d_ff_sim.log and wave.vcd
✅ d_ff simulation PASSED.

--- Simulating Clock Divider (clk_divider.v) ---
Compiling clk_divider.v and clk_divider_tb.v...
Compilation successful. Running simulation...
Simulation for clk_divider completed successfully.
Generated clk_divider_sim.log and wave.vcd
✅ clk_divider simulatio

### Explanation of Verilog Script Steps:

1.  **`echo "--- Simulating ... ---"`**: Provides clear headers for each simulation block.
2.  **`VERILOG_MODULE_NAME="..."`**: Defines a shell variable for the current module name, making it easy to reuse the script structure for different modules.
3.  **`iverilog -o <output_executable> <rtl_file.v> <testbench_file.v> > <compile_log.log> 2>&1`**:
    * `iverilog`: The Icarus Verilog compiler command.
    * `-o <output_executable>`: Specifies the name of the executable file that will be generated after successful compilation.
    * `<rtl_file.v> <testbench_file.v>`: The input Verilog source files. Both the design and its testbench are compiled together.
    * `> <compile_log.log> 2>&1`: This redirects all standard output (`stdout`) and standard error (`stderr`) from the `iverilog` command into a file named `<compile_log.log>`. This is crucial for debugging compilation errors.
4.  **`if [ $? -eq 0 ]; then ... else ... fi`**: This is a standard bash conditional statement.
    * `$?`: This special shell variable holds the exit status of the *last executed command*. A value of `0` typically indicates success, while any non-zero value indicates an error.
    * This block checks if the `iverilog` command (compilation) was successful.
5.  **`./<output_executable> > <sim_log.log> 2>&1`**:
    * `./<output_executable>`: Executes the compiled simulation.
    * The `testbench.v` contains `$dumpfile("wave.vcd");` and `$dumpvars(0, <module_name>);` which automatically generate the `wave.vcd` file.
    * `> <sim_log.log> 2>&1`: Redirects all standard output and standard error from the simulation run to a file named `<sim_log.log>`. This log will contain any `$display` messages from your testbench and runtime errors.
6.  **Success/Failure Messages**: `echo` statements are used to inform the user about the outcome of each compilation and simulation step, including where to find detailed logs.

## VHDL Simulation with GHDL

GHDL is a free VHDL simulator that can compile and execute VHDL code. It follows the IEEE 1076 standard. GHDL typically uses a two-step process: `analyze` to compile individual VHDL files into a working library, and `elaborate` to create an executable simulation from the analyzed units, followed by `run`.

### `%%bash` Script for VHDL Simulation

The following script automates the GHDL compilation and simulation process for VHDL designs.


In [ ]:
%%bash

# --- VHDL Simulation for T Flip-Flop ---
echo "--- Simulating T Flip-Flop (t_ff.vhdl) ---"

# Define the entity name for consistent log filenames and elaboration
VHDL_ENTITY_NAME="t_ff"
VHDL_TESTBENCH_ENTITY_NAME="${VHDL_ENTITY_NAME}_tb" # Testbench entity name is typically <entity_name>_tb

# Analyze the VHDL RTL file
# -a: Analyze
# >&: redirects both stdout and stderr to the log file
echo "Analyzing ${VHDL_ENTITY_NAME}.vhdl..."
ghdl -a ${VHDL_ENTITY_NAME}.vhdl > ${VHDL_ENTITY_NAME}_analyze_rtl.log 2>&1
COMPILE_STATUS_RTL=$?

# Analyze the VHDL Testbench file
echo "Analyzing ${VHDL_TESTBENCH_ENTITY_NAME}.vhdl..."
ghdl -a ${VHDL_TESTBENCH_ENTITY_NAME}.vhdl > ${VHDL_ENTITY_NAME}_analyze_tb.log 2>&1
COMPILE_STATUS_TB=$?

# Check if analysis was successful for both
if [ ${COMPILE_STATUS_RTL} -eq 0 ] && [ ${COMPILE_STATUS_TB} -eq 0 ]; then
    echo "Analysis successful. Elaborating and running simulation..."
    # Elaborate the testbench (creates the executable simulation)
    # -e: Elaborate
    ghdl -e ${VHDL_TESTBENCH_ENTITY_NAME} > ${VHDL_ENTITY_NAME}_elaborate.log 2>&1
    ELABORATE_STATUS=$?

    if [ ${ELABORATE_STATUS} -eq 0 ]; then
        # Run the simulation
        # -r: Run simulation
        # --wave=<filename.vcd>: specifies the VCD file name (GHDL uses .vcd by default, but good to be explicit)
        # --stop-time=<time>: stops the simulation after a specified duration (optional, but good for controlled runs)
        ghdl -r ${VHDL_TESTBENCH_ENTITY_NAME} --wave=${VHDL_ENTITY_NAME}_wave.vcd --stop-time=100ns > ${VHDL_ENTITY_NAME}_sim.log 2>&1
        SIM_STATUS=$?

        if [ ${SIM_STATUS} -eq 0 ]; then
            echo "Simulation for ${VHDL_ENTITY_NAME} completed successfully."
            echo "Generated ${VHDL_ENTITY_NAME}_sim.log and ${VHDL_ENTITY_NAME}_wave.vcd"
            echo "You can download ${VHDL_ENTITY_NAME}_wave.vcd and view it with GTKWave."
            echo "✅ ${VHDL_ENTITY_NAME} simulation PASSED."
        else
            echo "Simulation for ${VHDL_ENTITY_NAME} FAILED. Check ${VHDL_ENTITY_NAME}_sim.log for details."
            echo "❌ ${VHDL_ENTITY_NAME} simulation FAILED."
        fi
    else
        echo "Elaboration for ${VHDL_ENTITY_NAME} FAILED. Check ${VHDL_ENTITY_NAME}_elaborate.log for details."
        echo "❌ ${VHDL_ENTITY_NAME} elaboration FAILED."
    fi
else
    echo "Analysis for ${VHDL_ENTITY_NAME} FAILED. Check ${VHDL_ENTITY_NAME}_analyze_rtl.log and ${VHDL_ENTITY_NAME}_analyze_tb.log for details."
    echo "❌ ${VHDL_ENTITY_NAME} analysis FAILED."
fi
echo ""

# --- VHDL Simulation for JK Flip-Flop ---
echo "--- Simulating JK Flip-Flop (jk_ff.vhdl) ---"
VHDL_ENTITY_NAME="jk_ff"
VHDL_TESTBENCH_ENTITY_NAME="${VHDL_ENTITY_NAME}_tb"
echo "Analyzing ${VHDL_ENTITY_NAME}.vhdl..."
ghdl -a ${VHDL_ENTITY_NAME}.vhdl > ${VHDL_ENTITY_NAME}_analyze_rtl.log 2>&1
COMPILE_STATUS_RTL=$?
echo "Analyzing ${VHDL_TESTBENCH_ENTITY_NAME}.vhdl..."
ghdl -a ${VHDL_TESTBENCH_ENTITY_NAME}.vhdl > ${VHDL_ENTITY_NAME}_analyze_tb.log 2>&1
COMPILE_STATUS_TB=$?
if [ ${COMPILE_STATUS_RTL} -eq 0 ] && [ ${COMPILE_STATUS_TB} -eq 0 ]; then
    echo "Analysis successful. Elaborating and running simulation..."
    ghdl -e ${VHDL_TESTBENCH_ENTITY_NAME} > ${VHDL_ENTITY_NAME}_elaborate.log 2>&1
    ELABORATE_STATUS=$?
    if [ ${ELABORATE_STATUS} -eq 0 ]; then
        ghdl -r ${VHDL_TESTBENCH_ENTITY_NAME} --wave=${VHDL_ENTITY_NAME}_wave.vcd --stop-time=100ns > ${VHDL_ENTITY_NAME}_sim.log 2>&1
        SIM_STATUS=$?
        if [ ${SIM_STATUS} -eq 0 ]; then
            echo "Simulation for ${VHDL_ENTITY_NAME} completed successfully."
            echo "Generated ${VHDL_ENTITY_NAME}_sim.log and ${VHDL_ENTITY_NAME}_wave.vcd"
            echo "✅ ${VHDL_ENTITY_NAME} simulation PASSED."
        else
            echo "Simulation for ${VHDL_ENTITY_NAME} FAILED. Check ${VHDL_ENTITY_NAME}_sim.log for details."
            echo "❌ ${VHDL_ENTITY_NAME} simulation FAILED."
        fi
    else
        echo "Elaboration for ${VHDL_ENTITY_NAME} FAILED. Check ${VHDL_ENTITY_NAME}_elaborate.log for details."
        echo "❌ ${VHDL_ENTITY_NAME} elaboration FAILED."
    fi
else
    echo "Analysis for ${VHDL_ENTITY_NAME} FAILED. Check ${VHDL_ENTITY_NAME}_analyze_rtl.log and ${VHDL_ENTITY_NAME}_analyze_tb.log for details."
    echo "❌ ${VHDL_ENTITY_NAME} analysis FAILED."
fi
echo ""

# --- VHDL Simulation for D Flip-Flop ---
echo "--- Simulating D Flip-Flop (d_ff.vhdl) ---"
VHDL_ENTITY_NAME="d_ff"
VHDL_TESTBENCH_ENTITY_NAME="${VHDL_ENTITY_NAME}_tb"
echo "Analyzing ${VHDL_ENTITY_NAME}.vhdl..."
ghdl -a ${VHDL_ENTITY_NAME}.vhdl > ${VHDL_ENTITY_NAME}_analyze_rtl.log 2>&1
COMPILE_STATUS_RTL=$?
echo "Analyzing ${VHDL_TESTBENCH_ENTITY_NAME}.vhdl..."
ghdl -a ${VHDL_TESTBENCH_ENTITY_NAME}.vhdl > ${VHDL_ENTITY_NAME}_analyze_tb.log 2>&1
COMPILE_STATUS_TB=$?
if [ ${COMPILE_STATUS_RTL} -eq 0 ] && [ ${COMPILE_STATUS_TB} -eq 0 ]; then
    echo "Analysis successful. Elaborating and running simulation..."
    ghdl -e ${VHDL_TESTBENCH_ENTITY_NAME} > ${VHDL_ENTITY_NAME}_elaborate.log 2>&1
    ELABORATE_STATUS=$?
    if [ ${ELABORATE_STATUS} -eq 0 ]; then
        ghdl -r ${VHDL_TESTBENCH_ENTITY_NAME} --wave=${VHDL_ENTITY_NAME}_wave.vcd --stop-time=100ns > ${VHDL_ENTITY_NAME}_sim.log 2>&1
        SIM_STATUS=$?
        if [ ${SIM_STATUS} -eq 0 ]; then
            echo "Simulation for ${VHDL_ENTITY_NAME} completed successfully."
            echo "Generated ${VHDL_ENTITY_NAME}_sim.log and ${VHDL_ENTITY_NAME}_wave.vcd"
            echo "✅ ${VHDL_ENTITY_NAME} simulation PASSED."
        else
            echo "Simulation for ${VHDL_ENTITY_NAME} FAILED. Check ${VHDL_ENTITY_NAME}_sim.log for details."
            echo "❌ ${VHDL_ENTITY_NAME} simulation FAILED."
        fi
    else
        echo "Elaboration for ${VHDL_ENTITY_NAME} FAILED. Check ${VHDL_ENTITY_NAME}_elaborate.log for details."
        echo "❌ ${VHDL_ENTITY_NAME} elaboration FAILED."
    fi
else
    echo "Analysis for ${VHDL_ENTITY_NAME} FAILED. Check ${VHDL_ENTITY_NAME}_analyze_rtl.log and ${VHDL_ENTITY_NAME}_analyze_tb.log for details."
    echo "❌ ${VHDL_ENTITY_NAME} analysis FAILED."
fi
echo ""

# --- VHDL Simulation for Clock Divider ---
echo "--- Simulating Clock Divider (clk_divider.vhdl) ---"

# Define the entity name for consistent log filenames and elaboration
VHDL_ENTITY_NAME="clk_divider_div2" # This should be the DUT entity name
VHDL_TESTBENCH_ENTITY_NAME="${VHDL_ENTITY_NAME}_tb" # This should be the testbench entity name

# --- Define a UNIQUE GHW filename ---
# GHDL's native waveform format. GTKWave can open .ghw files directly.
GWH_FILENAME="${VHDL_ENTITY_NAME}_wave.ghw"

# --- VERIFY FILE EXISTENCE ---
echo "Verifying VHDL file existence..."
if [ ! -f "${VHDL_ENTITY_NAME}.vhdl" ]; then
    echo "ERROR: File '${VHDL_ENTITY_NAME}.vhdl' not found in the current directory."
    echo "Please ensure '${VHDL_ENTITY_NAME}.vhdl' is in the same directory as this script."
    echo "❌ File check FAILED."
    exit 1
fi
if [ ! -f "${VHDL_TESTBENCH_ENTITY_NAME}.vhdl" ]; then
    echo "ERROR: File '${VHDL_TESTBENCH_ENTITY_NAME}.vhdl' not found in the current directory."
    echo "Please ensure '${VHDL_TESTBENCH_ENTITY_NAME}.vhdl' is in the same directory as this script."
    echo "❌ File check FAILED."
    exit 1
fi
echo "VHDL files found."

# --- CLEAN PREVIOUS COMPILATION ARTIFACTS ---
echo "Cleaning existing 'work' library and previous GHW file..."
ghdl --remove > /dev/null 2>&1
rm -f "${GWH_FILENAME}" > /dev/null 2>&1 # Remove the old GHW file
echo "Cleaned 'work' library and removed old ${GWH_FILENAME}."

# --- ANALYSIS (Compilation) ---

# No need to compile GHDL VCD package anymore!
# Just compile your DUT and Testbench in the correct order.

# Analyze the VHDL RTL file FIRST. This is critical for component instantiation.
echo "Analyzing ${VHDL_ENTITY_NAME}.vhdl..."
ghdl -a ${VHDL_ENTITY_NAME}.vhdl > ${VHDL_ENTITY_NAME}_analyze_rtl.log 2>&1
COMPILE_STATUS_RTL=$?

if [ ${COMPILE_STATUS_RTL} -ne 0 ]; then
    echo "Analysis for ${VHDL_ENTITY_NAME} FAILED. Check ${VHDL_ENTITY_NAME}_analyze_rtl.log for details."
    echo "❌ ${VHDL_ENTITY_NAME} analysis FAILED."
    cat ${VHDL_ENTITY_NAME}_analyze_rtl.log
    exit 1
fi

# Analyze the VHDL Testbench file SECOND.
echo "Analyzing ${VHDL_TESTBENCH_ENTITY_NAME}.vhdl..."
ghdl -a ${VHDL_TESTBENCH_ENTITY_NAME}.vhdl > ${VHDL_ENTITY_NAME}_analyze_tb.log 2>&1
COMPILE_STATUS_TB=$?

if [ ${COMPILE_STATUS_TB} -ne 0 ]; then
    echo "Analysis for ${VHDL_TESTBENCH_ENTITY_NAME} FAILED. Check ${VHDL_ENTITY_NAME}_analyze_tb.log for details."
    echo "❌ ${VHDL_TESTBENCH_ENTITY_NAME} analysis FAILED."
    cat ${VHDL_ENTITY_NAME}_analyze_tb.log
    exit 1
fi

echo "Analysis successful."

# --- ELABORATION ---
echo "Elaborating design for ${VHDL_TESTBENCH_ENTITY_NAME}..."
ghdl -e ${VHDL_TESTBENCH_ENTITY_NAME} > ${VHDL_ENTITY_NAME}_elaborate.log 2>&1
ELABORATE_STATUS=$?

if [ ${ELABORATE_STATUS} -ne 0 ]; then
    echo "Elaboration for ${VHDL_TESTBENCH_ENTITY_NAME} FAILED. Check ${VHDL_ENTITY_NAME}_elaborate.log for details."
    echo "❌ ${VHDL_ENTITY_NAME} elaboration FAILED."
    cat ${VHDL_ENTITY_NAME}_elaborate.log
    exit 1
fi

echo "Elaboration successful."

# --- SIMULATION ---
echo "Running simulation for ${VHDL_TESTBENCH_ENTITY_NAME} and dumping to ${GWH_FILENAME}..."
# Use the dynamic GHW_FILENAME here with --wave
ghdl -r ${VHDL_TESTBENCH_ENTITY_NAME} --wave="${GWH_FILENAME}" --stop-time=100ns > ${VHDL_ENTITY_NAME}_sim.log 2>&1
SIM_STATUS=$?

if [ ${SIM_STATUS} -eq 0 ]; then
    echo "Simulation for ${VHDL_ENTITY_NAME} completed successfully."
    echo "Generated ${VHDL_ENTITY_NAME}_sim.log and ${GWH_FILENAME}"
    echo "You can download ${GWH_FILENAME} and view it with GTKWave."
    echo "✅ ${VHDL_ENTITY_NAME} simulation PASSED."
else
    echo "Simulation for ${VHDL_ENTITY_NAME} FAILED. Check ${VHDL_ENTITY_NAME}_sim.log for details."
    echo "❌ ${VHDL_ENTITY_NAME} simulation FAILED."
    cat ${VHDL_ENTITY_NAME}_sim.log
fi

echo ""

# --- VHDL Simulation for 2-bit Synchronous Counter ---
echo "--- Simulating 2-bit Synchronous Counter (sync_counter_2bit.vhdl) ---"
VHDL_ENTITY_NAME="sync_counter_2bit"
VHDL_TESTBENCH_ENTITY_NAME="${VHDL_ENTITY_NAME}_tb"
echo "Analyzing ${VHDL_ENTITY_NAME}.vhdl..."
ghdl -a ${VHDL_ENTITY_NAME}.vhdl > ${VHDL_ENTITY_NAME}_analyze_rtl.log 2>&1
COMPILE_STATUS_RTL=$?
echo "Analyzing ${VHDL_TESTBENCH_ENTITY_NAME}.vhdl..."
ghdl -a ${VHDL_TESTBENCH_ENTITY_NAME}.vhdl > ${VHDL_ENTITY_NAME}_analyze_tb.log 2>&1
COMPILE_STATUS_TB=$?
if [ ${COMPILE_STATUS_RTL} -eq 0 ] && [ ${COMPILE_STATUS_TB} -eq 0 ]; then
    echo "Analysis successful. Elaborating and running simulation..."
    ghdl -e ${VHDL_TESTBENCH_ENTITY_NAME} > ${VHDL_ENTITY_NAME}_elaborate.log 2>&1
    ELABORATE_STATUS=$?
    if [ ${ELABORATE_STATUS} -eq 0 ]; then
        ghdl -r ${VHDL_TESTBENCH_ENTITY_NAME} --wave=${VHDL_ENTITY_NAME}_wave.vcd --stop-time=100ns > ${VHDL_ENTITY_NAME}_sim.log 2>&1
        SIM_STATUS=$?
        if [ ${SIM_STATUS} -eq 0 ]; then
            echo "Simulation for ${VHDL_ENTITY_NAME} completed successfully."
            echo "Generated ${VHDL_ENTITY_NAME}_sim.log and ${VHDL_ENTITY_NAME}_wave.vcd"
            echo "✅ ${VHDL_ENTITY_NAME} simulation PASSED."
        else
            echo "Simulation for ${VHDL_ENTITY_NAME} FAILED. Check ${VHDL_ENTITY_NAME}_sim.log for details."
            echo "❌ ${VHDL_ENTITY_NAME} simulation FAILED."
        fi
    else
        echo "Elaboration for ${VHDL_ENTITY_NAME} FAILED. Check ${VHDL_ENTITY_NAME}_elaborate.log for details."
        echo "❌ ${VHDL_ENTITY_NAME} elaboration FAILED."
    fi
else
    echo "Analysis for ${VHDL_ENTITY_NAME} FAILED. Check ${VHDL_ENTITY_NAME}_analyze_rtl.log and ${VHDL_ENTITY_NAME}_analyze_tb.log for details."
    echo "❌ ${VHDL_ENTITY_NAME} analysis FAILED."
fi
echo ""

--- Simulating T Flip-Flop (t_ff.vhdl) ---
Analyzing t_ff.vhdl...
Analyzing t_ff_tb.vhdl...
Analysis successful. Elaborating and running simulation...
Simulation for t_ff completed successfully.
Generated t_ff_sim.log and t_ff_wave.vcd
You can download t_ff_wave.vcd and view it with GTKWave.
✅ t_ff simulation PASSED.

--- Simulating JK Flip-Flop (jk_ff.vhdl) ---
Analyzing jk_ff.vhdl...
Analyzing jk_ff_tb.vhdl...
Analysis successful. Elaborating and running simulation...
Simulation for jk_ff completed successfully.
Generated jk_ff_sim.log and jk_ff_wave.vcd
✅ jk_ff simulation PASSED.

--- Simulating D Flip-Flop (d_ff.vhdl) ---
Analyzing d_ff.vhdl...
Analyzing d_ff_tb.vhdl...
Analysis successful. Elaborating and running simulation...
Simulation for d_ff completed successfully.
Generated d_ff_sim.log and d_ff_wave.vcd
✅ d_ff simulation PASSED.

--- Simulating Clock Divider (clk_divider.vhdl) ---
Verifying VHDL file existence...
VHDL files found.
Cleaning existing 'work' library and previo

### Explanation of VHDL Script Steps:

1.  **`echo "--- Simulating ... ---"`**: Similar to Verilog, provides clear separation.
2.  **`VHDL_ENTITY_NAME="..."` and `VHDL_TESTBENCH_ENTITY_NAME="..."`**: Defines shell variables for the RTL entity and its corresponding testbench entity. This is crucial as GHDL operates on entity names.
3.  **`ghdl -a <rtl_file.vhdl> > <analyze_rtl_log.log> 2>&1`**:
    * `ghdl -a`: This command analyzes (compiles) the VHDL source file. Each VHDL file is typically analyzed separately.
    * The output and error are redirected to an analysis log specific to the RTL file.
    * `COMPILE_STATUS_RTL=$?`: Stores the exit status for later checking.
4.  **`ghdl -a <testbench_file.vhdl> > <analyze_tb_log.log> 2>&1`**:
    * Analyzes the testbench VHDL file similarly.
    * `COMPILE_STATUS_TB=$?`: Stores the exit status for the testbench analysis.
5.  **`if [ ${COMPILE_STATUS_RTL} -eq 0 ] && [ ${COMPILE_STATUS_TB} -eq 0 ]; then ... else ... fi`**: Checks if *both* the RTL and testbench analysis steps were successful.
6.  **`ghdl -e ${VHDL_TESTBENCH_ENTITY_NAME} > <elaborate_log.log> 2>&1`**:
    * `ghdl -e`: This command elaborates the specified top-level entity (which should be your testbench entity). Elaboration links all analyzed design units and prepares the simulation model.
    * Output and error are redirected to an elaboration log.
    * `ELABORATE_STATUS=$?`: Stores the exit status of the elaboration.
7.  **`ghdl -r ${VHDL_TESTBENCH_ENTITY_NAME} --wave=<filename.vcd> --stop-time=100ns > <sim_log.log> 2>&1`**:
    * `ghdl -r`: Runs the simulation of the elaborated design.
    * `--wave=<filename.vcd>`: Tells GHDL to dump waveform data into the specified VCD file. This creates the visual waveform for analysis. It's important to use a unique name for each module's VCD to avoid overwriting.
    * `--stop-time=<time>`: (Optional but recommended) Specifies a maximum simulation time. This prevents runaway simulations if your testbench doesn't have a natural `$finish` or `wait;` condition.
    * Output and error are redirected to a simulation log.
    * `SIM_STATUS=$?`: Stores the exit status of the simulation run.
8.  **Success/Failure Messages**: `echo` statements provide comprehensive feedback on each stage (analysis, elaboration, simulation) and guide the user to the relevant log files for debugging.

After running these scripts, you will find `.log` files containing compilation and simulation outputs, and `.vcd` files for waveform viewing. These files can be downloaded from the Colab environment using the file browser on the left sidebar.

# Archiving and Downloading Your Digital Logic Design Files in Google Colab

After successfully generating RTL code, testbenches, and running simulations, managing the numerous output files (`.v`, `.vhdl`, `.log`, `.vcd`) becomes important. This chapter provides a convenient Python script designed for Google Colab environments to efficiently archive these files into a single zip archive and then facilitate its download.

## The Need for Archiving and Easy Download

In a cloud-based environment like Google Colab, files are stored temporarily. While you can manually download individual files, consolidating them into a zip archive offers several advantages:

* **Organization:** Keeps all related design and simulation outputs together.
* **Efficiency:** Downloads multiple files as a single package, saving time and effort.
* **Portability:** Easily transfer your entire design workspace to another environment or for local storage.

The Python script below automates this process, creating a timestamped zip file and providing the necessary Colab-specific code to download it directly to your local machine.

## Archiving Simulation Results

The following Python script will identify all relevant design and simulation files in your current Colab working directory and compress them into a new zip file. The zip file will be named dynamically using the current date and time to ensure uniqueness.


In [ ]:
import datetime
import zipfile
import os
from IPython.display import display, Javascript

# 1. Create a dynamic filename with current timestamp
# Get the current time
current_time = datetime.datetime.now()
# Format the time for the filename (e.g., 20250731_105630)
timestamp_str = current_time.strftime("%Y%m%d_%H%M%S")
# Construct the filename
filename = f"ai_for_dld_0401_{timestamp_str}"
zip_file_name = f"{filename}.zip"

print(f"Preparing to create zip file: {zip_file_name}")

# 2. Identify and zip all generated files
# Define the file extensions to include in the zip
file_extensions = ('.v', '.vhdl', '.log', '.vcd')
files_to_zip = []

# Walk through the current directory and find files with specified extensions
for root, _, files in os.walk('.'):
    for file in files:
        # Check if the file has one of the desired extensions
        if file.endswith(file_extensions):
            files_to_zip.append(os.path.join(root, file))

# Create the zip archive
try:
    with zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for file in files_to_zip:
            # Add file to zip, preserving its path relative to the current directory
            zipf.write(file, arcname=file)
    print(f"✅ Successfully created zip file: {zip_file_name}")
except Exception as e:
    print(f"❌ Error creating zip file: {e}")


Preparing to create zip file: ai_for_dld_0401_20250731_054643.zip
✅ Successfully created zip file: ai_for_dld_0401_20250731_054643.zip


### Explanation of the Archiving Script:

1.  **Import Libraries:**
    * `datetime`: Used to get the current date and time for the unique filename.
    * `zipfile`: Python's built-in module for creating and working with zip archives.
    * `os`: Provides functions for interacting with the operating system, such as listing files in a directory (`os.walk`).
2.  **Dynamic Filename Generation:**
    * `datetime.datetime.now()`: Fetches the current date and time.
    * `strftime("%Y%m%d_%H%M%S")`: Formats the datetime object into a string (e.g., `20250731_105630`), making it suitable for filenames.
    * An f-string is used to construct the final `zip_file_name`.
3.  **File Identification:**
    * `file_extensions`: A tuple defining the types of files to be included in the archive.
    * `os.walk('.')`: Iterates through the current directory (`.`) and its subdirectories, yielding the root directory, subdirectories, and filenames.
    * The script checks if each file's name ends with any of the specified extensions using `file.endswith()`.
4.  **Zip Archive Creation:**
    * `zipfile.ZipFile(zip_file_name, 'w', zipfile.ZIP_DEFLATED)`: Opens a new zip file in write mode (`'w'`). `zipfile.ZIP_DEFLATED` specifies that the files should be compressed.
    * `zipf.write(file, arcname=file)`: Adds each identified file to the zip archive. `arcname=file` ensures that the file's path within the zip matches its relative path in the current directory.
    * A `try-except` block is used for basic error handling during zip creation.
5.  **Success Message:** A clear message is printed to confirm the successful creation of the zip file.

## Downloading the Archive

Once the zip file is created, you can use Google Colab's built-in `files.download` utility to transfer it from the Colab environment to your local machine.


In [ ]:
from google.colab import files

# Provide the name of the zip file created by the previous script
# Ensure this matches the 'zip_file_name' variable from the previous cell
# You might need to manually update this if you run the zipping script multiple times
# and want to download a specific version.
# For example, if the previous script created 'ai_for_dld_0401_20250731_105630.zip', use that name here.
# To make it dynamic, we'll reuse the 'zip_file_name' variable if this cell is run after the previous one.

# If you run this cell independently, uncomment and set the exact filename:
# zip_file_name = "ai_for_dld_0401_YYYYMMDD_HHMMSS.zip" # Replace with actual filename

print(f"Attempting to download {zip_file_name}...")
try:
    files.download(zip_file_name)
    print(f"Initiated download for {zip_file_name}. Check your browser's downloads.")
except Exception as e:
    print(f"❌ Error initiating download: {e}")
    print("Please ensure the zip file exists and the filename is correct.")


Attempting to download ai_for_dld_0401_20250731_054643.zip...


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Initiated download for ai_for_dld_0401_20250731_054643.zip. Check your browser's downloads.


### Explanation of the Downloading Script:

1.  **`from google.colab import files`**: Imports the necessary module from the Colab environment.
2.  **`files.download(zip_file_name)`**: This function triggers the download of the specified file directly to your browser's default download location.
3.  **Dynamic Filename (Re-use):** The script assumes that the `zip_file_name` variable from the previous cell (the archiving script) is still in scope. If you run this cell in a fresh session or after modifying the archiving script, you might need to manually set the `zip_file_name` variable to the exact name of the zip file you wish to download.
4.  **Error Handling:** A `try-except` block is included to catch potential errors during the download process, such as the file not being found.

By executing these two Python code blocks in sequence within your Google Colab notebook, you can effectively manage, archive, and retrieve all your digital logic design and simulation artifacts.